In [1]:
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc # 폰트 세팅을 위한 모듈 추가

warnings.filterwarnings('ignore')

font_path = "malgun.ttf" # 사용할 폰트명 경로 삽입
font = font_manager.FontProperties(fname = font_path).get_name()
rc('font', family = font)

In [2]:
foodSet = pd.read_csv('./Final_Cluster_Result.csv', sep=',')

In [3]:
# 섭취 음식 입력
# ex) 1끼 먹을 음식을 제한적인 갯수로(가령 5개 정도) 한정하여 입력하도록 조치하여 이를 일일권장량 기준의 1/3로 환산하여  
# 각 영양소에 대한 부족분을 파악하고, 이에 따라서 임력한 음식(기호 음식)을 중심으로 후보군 내에서 적합한 음식을 필터링하여 추천 음식을 선정함

# 구체적인 방안) 한끼 식사에 대한 A ~ E까지 음식이 입력되었다고 전제하고, 권장량에 충족되면 입력 그대로 진행하고 그렇지 않으면, 부족한 영양소를 A ~ E
# 음식 순으로 기호에 맞게 유사도가 가까운 음식 후보군 중에서 미달된 영양 충족하는 음식을 선정하여 추천함 

In [4]:
inputFoods = {} # 음식(key) : 수량(value)

inputFoodA = foodSet.loc[foodSet['식품명'].str.contains('햄버거')].iloc[0]
inputFoodA
inputFoods[inputFoodA['식품명']] = 1

In [5]:
inputFoodB = foodSet.loc[foodSet['식품명'].str.contains('햇반저단백밥')].iloc[0]
inputFoodB
inputFoods[inputFoodB['식품명']] = 1

In [6]:
inputFoodB = foodSet.loc[foodSet['식품명'].str.contains('펩시콜라')].iloc[0]
inputFoodB
inputFoods[inputFoodB['식품명']] = 1

In [7]:
inputFoodC = foodSet.loc[foodSet['식품명'].str.contains('감자칩')].iloc[2]
inputFoodC
inputFoods[inputFoodC['식품명']] = 1

In [8]:
inputFoodD = foodSet.loc[foodSet['식품명'].str.contains('열라면')].iloc[3]
inputFoodD
inputFoods[inputFoodD['식품명']] = 1

In [9]:
inputFoods

{'빵_노랑햄버거빵': 1,
 '환자용 식품_햇반저단백밥': 1,
 '탄산음료_펩시콜라': 1,
 '일반과자_트러플 감자칩': 1,
 '라면_마열라면Cup': 1}

---

In [10]:
import os
import cx_Oracle

# DB 한글 설정에 따른 한글 지원
# os.putenv('NLS_LANG', 'KOREAN_KOREA.KO16MSWIN949');
os.putenv('NLS_LANG', '.UTF8')

conn = cx_Oracle.connect('food', 'food', 'localhost:1521/xe')
cursor = conn.cursor()

In [11]:
# 성별과 나이대로 식약처 권장 영양 정보 조회
def findByGenderAndAge(gender, ageRange):

    sql_str = "select * from nutri_std_tbl where gender=:gender and age=:ageRange"
    records = cursor.execute(sql_str, [gender, ageRange])
    record = records.fetchone()

    # print('레코드 : ', record)

    # ['NUM', 'GENDER', 'AGE', 'CARBOHYDRATE', 'PROTEIN', 
    #  'SUGAR', 'NATRIUM', 'CHOLESTEROL', 'FAT', 'FATTY_ACID', 'TRANS_FATTY_ACID']
   
    return record

# 특정 나이의 나이대를 판정
# ex) 20 => 19~29
def calcAgeBand(age):
		
    result = ""
    
    ageRanges = ("6~8", "9~11", "12~14", "15~18", "19~29", "30~49", "50~64", "65~74", "75~")

    # print(ageRanges)
    
    # 리스트의 나이 최소 나이 추출하여 나이대의 최솟값의 리스트 생성
    # 나이대의 최솟값으로  6,9,12,15,19,30, .....
    ageRangeMins = [int(ageRange.split("~")[0]) for ageRange in ageRanges]
        
    # print(ageRangeMins)
    
    # 해당되는 나이(가령 20) 보다 이하 인(작거나 같은) 나이 중에서 최댓값을 구함 => 19
    tempAge = max([ageRangeMin for ageRangeMin in ageRangeMins if ageRangeMin <= age])
    
    # print("tempAge :", tempAge)
    
    # 나이대 계산
    result = [ageRange for ageRange in ageRanges if str(tempAge) in ageRange]

    # print(result[0])
        
    return result[0]

# 성별과 나이대로 식약처 권장 영양 정보 조회
def findByGenderAndAge(gender, ageRange):

    sql_str = "select * from nutri_std_tbl where gender=:gender and age=:ageRange"
    records = cursor.execute(sql_str, [gender, ageRange])
    record = records.fetchone()

    # print('레코드 : ', record)

    # ['NUM', 'GENDER', 'AGE', 'ENERGY', 'CARBOHYDRATE', 'PROTEIN', 
    #  'SUGAR', 'NATRIUM', 'CHOLESTEROL', 'FAT', 'FATTY_ACID', 'TRANS_FATTY_ACID']
   
    return record

# findByGenderAndAge('남', calcAgeBand(20))
# (5,  '남', '19~29',   2600,         130,          65,     '10~20', 1500,        300,       '15~30',          7,            1)
# 번호, 성별, 연령대, 권장열량(kcal), 탄수화물(g), 단백질(g), 지방(g), 당류(g),  나트륨(mg), 콜레스테롤(mg),  포화지방산(g), 트랜스지방(g)

In [12]:
# 성별, 나이, 신장, 체중, PA(Physical Activity:신체활동량)
# 영양권장량 (Recommended Dietary Allowance, RDA)

# 에너지 요구량 추정치(ESTIMATED ENERGY REQUIREMENT (EER))

# 관련 논문(근거) : https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7004509/
# https://globalrph.com/medcalcs/estimated-energy-requirement-eer-equation/
# https://www.diabetes.or.kr/general/dietary/dietary_02.php?con=2
# https://www.eatforhealth.gov.au/nutrition-calculators/daily-energy-requirements-calculator

# PA(남성) : 1.1(저활동적), 1.48(매우 활동적)
# 원문) PA = 1.0 (sedentary), 1.11 (low active), 1.25 (active), or 1.48 (very active).

# PA(여성) : 1.2(저활동적), 1.45(매우 활동적)
# 원문) PA = 1.0 (sedentary), 1.12 (low active), 1.27 (active), or 1.45 (very active).

# PA 기준안 그림 포함)
# https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7004509/
# https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7004509/bin/JENB_2019_v23n4_1_f001.jpg

# 한국인 영양섭취 기준 : https://www.mohw.go.kr/boardDownload.es?bid=0019&list_no=335863&seq=1

# 영양권장량(RDA) 계산
# weight(체중) : kg, 
# height(신장) : m
# EER(에너지 요구량 추정치(ESTIMATED ENERGY REQUIREMENT (EER))) : kcal
# disease(질병) : 고혈압, 당뇨, 고지혈증, 비만
def calculateRDA(disease, gender, age, height, weight, PA=1):

    #recom = [] # 권장량

    # 에너지 추정량(kcal) : ESTIMATED ENERGY REQUIREMENT (EER) EQUATION
    # 관련 논문(근거) : https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7004509/
    # https://globalrph.com/medcalcs/estimated-energy-requirement-eer-equation/
    EER = 0

    if gender == '남':
        EER = 622 - 9.53 * age + PA * (15.91 * weight + 536.6 * height)
    elif gender == '여':    
        EER = 354 - 6.91 * age + PA * (9.36 * weight + 726 * height)

    # 식약처 기준량 조회
    # 번호, 성별, 연령대, 권장열량(kcal), 탄수화물(g), 단백질(g), 지방(g), 당류(g),  나트륨(mg), 콜레스테롤(mg),  포화지방산(g), 트랜스지방(g)
    age_arange = calcAgeBand(age)
    print("연령대 : ", age_arange)
    
    nutri_tot = findByGenderAndAge(gender, age_arange)
    nutri_recomm = list(nutri_tot)
    
    # 불필요성분 제거 ex) 번호, 성별, 연령대
    nutri_recomm = nutri_recomm[3:]
    
    # 권장열량(kcal), 탄수화물(g), 단백질(g), 지방(g), 당류(g),  나트륨(mg), 콜레스테롤(mg),  포화지방산(g), 트랜스지방(g)
    print("권장 섭취량 (일반) : ", nutri_recomm)

    # 관련 근거 : https://www.ncbi.nlm.nih.gov/books/NBK218738/

    # 참고) 연도별 우리 국민의 1일 나트륨평균 섭취량 추이
    # (2022.12.31. 기준, 단위: ㎎/day, 출처: 국민건강영양조사, 질병관리청)
    
    # 연도           2011 2012  2013  2014  2015  2016  2017  2018  2019  2020  2021
    # 나트륨 섭취량 4,831 4,583 4,027 3,890 3,890 3,669 3,478 3,274 3,289 3,220 3,080

    # 관련 근거 : 2020 한국인 영양소 섭취기준 활용 자료 개발
    # https://e-jnh.org/pdf/10.4163/jnh.2022.55.1.21
    
    # 질병이 있을 경우  ex) disease(질병) : 고혈압, 당뇨, 고지혈증, 비만

    # 참고) 고혈압의 경우 나트륨 권장량이 3700ml로 되어 있는 것은 권장량과 다르나 최근 식약처의 충분량 섭취 제개정 기준 변화로 그대로 수용
    # 나트륨 관련 규정 : 2020 한국인 영양소 섭취기준 <보건복지부, 한국영양학회> 1. 한국인 영양소 섭취기준 제·개정 요약 p. 27
    # https://www.kns.or.kr/FileRoom/FileRoom_view.asp?idx=108&BoardID=Kdr
    
    # "당뇨 환자의 탄수화물 섭취량이 많은 이유 !"
    # 당뇨에 따른 탄수화물 섭취량 관련 논문)
    #
    # "당뇨병 환자는 일반적으로 총 열량의 50~60%를 탄수화물로 섭취하도록 권고하고 있으며, 탄수화물, 단백질, 
    # 지방 섭취량은 식습관, 기호도, 치료목표 등을 고려하여 개별화할 수 있다"
    # 논문 인용) 탄수화물계산을 활용한 임상영양요법 : 김미라 분당서울대학교병원 영양실
    # : https://www.e-jkd.org/upload/pdf/jkd-2022-23-2-133.pdf

    # 참고) 당뇨 환자 일일 필요 열량 측정 프로그램 : https://www.diabetes.or.kr/general/dietary/dietary_02.php?con=2

    if (disease == '고혈압'):
        nutri_recomm[5] = 3700 # 단위 mg
        print("nutri_recomm[5] (나트륨) : ", nutri_recomm[5])
    
    if (disease == '당뇨'):
        nutri_recomm[1] = EER * 0.7 // 4 # 단위 g        
        print("nutri_recomm[1] (탄수화물) : ", nutri_recomm[1]) # 탄수화물
    
    if (disease == '고지혈증'):
        nutri_recomm[6] = EER * 0.07 // 9 # 단위 g
        print("nutri_recomm[6] (포화지방산) : ", nutri_recomm[6]) # 포화지방산

    print("권장 섭취량(유병자 적용 후) : ", nutri_recomm)

    return nutri_recomm

In [13]:
#calculateRDA('', '여', 20, 1.6, 50, 1.25) 

In [14]:
#calculateRDA('고혈압', '남', 45, 1.85, 100, 1.11) 

In [15]:
#calculateRDA('당뇨', '여', 50, 1.75, 90, 1.0) 

In [16]:
#calculateRDA('고지혈증', '남', 20, 1.7, 70, 1.25)

---

In [17]:
# 전체 입력 음식들에 대한 영양소 현황 집계
totFoods = pd.DataFrame(columns=['식품코드', '식품명', '에너지(kcal)', '탄수화물(g)', '단백질(g)', '지방(g)', '포화지방산(g)', '나트륨(mg)']);
print(type(totFoods))

count = 0

for foodName in inputFoods:
    
    foodRow = foodSet.loc[foodSet['식품명'] == foodName].drop_duplicates()

    # print("행수 : ", len(foodRow))
    
    if len(foodRow) == 2:
        foodRow = foodRow.head(1)

    print("수량 : ", inputFoods[foodName])

    #foodRow = foodRow * inputFoods[foodName] # 식품 * 수량 = 식품별 영양 사항
    
    foodRow['에너지(kcal)'] *= inputFoods[foodName]
    foodRow['탄수화물(g)'] *= inputFoods[foodName]
    foodRow['단백질(g)'] *= inputFoods[foodName]
    foodRow['지방(g)'] *= inputFoods[foodName]
    foodRow['포화지방산(g)'] *= inputFoods[foodName]
    foodRow['나트륨(mg)'] *= inputFoods[foodName]
    
    totFoods = pd.concat([totFoods, foodRow], ignore_index=True)
    
totFoods

<class 'pandas.core.frame.DataFrame'>
수량 :  1
수량 :  1
수량 :  1
수량 :  1
수량 :  1


,식품코드,식품명,에너지(kcal),탄수화물(g),단백질(g),지방(g),포화지방산(g),나트륨(mg),Unnamed: 0,대표식품명,식품대분류명,식품중분류명,식품소분류명,clusterLabel
0,P101-405000400-2833,빵_노랑햄버거빵,245,40.00,4.00,4.00,1.00,5.0,9683.0,빵,과자류·빵류 또는 떡류,해당없음,빵류,0.0
1,P111-000020100-0013,환자용 식품_햇반저단백밥,156,37.22,0.22,0.61,0.00,6.0,44566.0,환자용 식품,특수의료용도식품,맞춤형 영양조제식품,선천성대사질환자용조제식품,1.0
2,P109-401040100-0554,탄산음료_펩시콜라,44,11.20,0.00,0.00,0.00,3.0,39171.0,탄산음료,음료류,탄산음료류,탄산음료,1.0
3,P101-106000100-2563,일반과자_트러플 감자칩,600,46.67,6.67,43.33,13.33,533.0,30294.0,일반과자,과자류·빵류 또는 떡류,해당없음,과자,2.0
4,P108-003000400-0341,라면_마열라면Cup,427,67.74,9.68,12.90,4.84,1645.0,6664.0,라면,면류,해당없음,유탕면,3.0


In [18]:
# 음식에 대한 분야별 총량 계산

totFoods['에너지(kcal)'].sum()
# totFoods.describe()

1472

In [19]:
# 일일권장 섭취량 조회
# 권장열량(kcal), 탄수화물(g), 단백질(g), 지방(g), 당류(g),  나트륨(mg), 콜레스테롤(mg),  포화지방산(g), 트랜스지방(g)
standardRDA = calculateRDA('암', '남', 75, 1.70, 60, 1.00)

disease = '암' # 병종 변수 추가

연령대 :  75~
권장 섭취량 (일반) :  [1900, 130, 60, '10~20', 1100, 300, '15~30', 7, 1]
권장 섭취량(유병자 적용 후) :  [1900, 130, 60, '10~20', 1100, 300, '15~30', 7, 1]


In [20]:
# 영양 결핍량 측정
# 에너지(kcal) 비교
# 영양 결핍분 집계 dataframe

nutriNeeds = {'에너지(kcal)' : [0], '탄수화물(g)' : [0], '단백질(g)' : [0], '지방(g)' : [0], '포화지방산(g)' : [0], '나트륨(mg)' : [0]}
nutriNeedsDF = pd.DataFrame(nutriNeeds, columns=['에너지(kcal)', '탄수화물(g)', '단백질(g)', '지방(g)', '포화지방산(g)', '나트륨(mg)'])

nutriNeedsDF['에너지(kcal)'] = totFoods['에너지(kcal)'].sum() - standardRDA[0]
nutriNeedsDF['탄수화물(g)']  = totFoods['탄수화물(g)'].sum() - standardRDA[1]
nutriNeedsDF['단백질(g)']    = totFoods['단백질(g)'].sum() - standardRDA[2]

#nutriNeedsDF['지방(g)']      = totFoods['지방(g)'].sum() - standardRDA[3]

# 지방 10~20
stdFat = standardRDA[3].split("~")
minFat = int(stdFat[0])
maxFat = int(stdFat[1])

#print("지방 : ", totFoods['지방(g)'].sum())

if totFoods['지방(g)'].sum() < minFat:
    nutriNeedsDF['지방(g)'] = minFat - totFoods['지방(g)'].sum()
elif totFoods['지방(g)'].sum() > maxFat:    
    nutriNeedsDF['지방(g)'] = totFoods['지방(g)'].sum() - maxFat

nutriNeedsDF['포화지방산(g)'] = totFoods['포화지방산(g)'].sum() - standardRDA[7]
nutriNeedsDF['나트륨(mg)'] = totFoods['나트륨(mg)'].sum() - standardRDA[5]

In [21]:
# 영양 성분 분석 결과
# 양수 이면 기준량에 대한 과잉분이 출력, 음수이면 기준량 대비 부족분으로 판단 
nutriNeedsDF

,에너지(kcal),탄수화물(g),단백질(g),지방(g),포화지방산(g),나트륨(mg)
0,-428,72.83,-39.43,40.84,12.17,1892.0


---

In [22]:
# foodSet.loc[0]['clusterLabel']
totFoods.iloc[0]['clusterLabel']

0.0

In [23]:
# foodSet 중에서 
# "대표식품명" 필드에서 "환자용 식품", "고령자용 영양조제식품", "체중조절용 조제식품" 및
# "식품소분류명" 필드에서 "당뇨"이면 "당뇨환자용 영양조제식품"
# 위주로 dataframe 재구성

patientFoodDF = foodSet

#print(len(patientFoodDF))
#print(patientFoodDF.head(1))

patientFoodDF = patientFoodDF.loc[patientFoodDF['대표식품명'] == '환자용 식품']

patientFoodDF = patientFoodDF.loc[patientFoodDF['대표식품명'] == '환자용 식품']
len(patientFoodDF)

# 당뇨 환자용 # 66개
diabetesFoodDF = patientFoodDF.loc[patientFoodDF['식품명'].str.contains('당뇨')]
print(len(diabetesFoodDF))
#print(diabetesFoodDF.head())

# 암 환자용 # 3개
cancerFoodDF = patientFoodDF.loc[patientFoodDF['식품소분류명'] == '암환자용 영양조제식품']
print(len(cancerFoodDF))
# print(cancerFoodDF.head())

# 고혈압 환자용 # 4개
hypertensionFoodDF = patientFoodDF.loc[patientFoodDF['식품소분류명'].str.contains('고혈압')]
print(len(hypertensionFoodDF))
# print(hypertensionFoodDF.head())


66
3
4


In [24]:
# 영양 결핍 부족분(값이 음수)에 따른 추천 음식 선정
# 영양 결핍 부족분(값이 음수)인 영양소 파악

#chosenDF= pd.DataFrame()

if disease == '' or disease == '고혈압' or disease == '고지혈증' : # 무병자 및 고혈압 환자, 고지혈증
    
    for nutri in nutriNeedsDF:
            
        if float(nutriNeedsDF[nutri]) < 0: # 결핍 영양소
        
            # 결핍 영양소에 따른 음식 검색
            # 입력된 음식들의 군집 범위 내에서 검색하되 첫번째 음식부터 검색
            print("부족 영양소(nutri) : ", nutri)
            print("영양소 부족분(절대값) : ", abs(float(nutriNeedsDF[nutri])))
    
            # foodSet에서 같은 클러스터의 권장 섭취분 해당 음식 검색
    
            # 과식 방지를 위해 부족분(ex. 단백질 -15.63 ==> 15.63)에 가장 가까운 값(최소값) 선정
            # 첫번째 입력 음식의 clusterLabel과 비교
            print("clusterLabel : ", totFoods.iloc[0]['clusterLabel'])
    
            # 먼저 동일한 clusterLabel을 선정 
            chosenDF = foodSet.loc[foodSet['clusterLabel'] == totFoods.iloc[0]['clusterLabel']]
            
            # 영양소 결핍분에 대한 충족 음식 조회 
            chosenDF = chosenDF.loc[foodSet[nutri] >= abs(float(nutriNeedsDF[nutri]))]
    
            print("클러스터내 추천 음식 존재 여부 : ", len(chosenDF))
            
            # 만약 환자에 해당된다면 환자에 최적화된 dataframe에서 검색        
            
            # if len(chosenDF) == 0:
            if chosenDF.empty == True:
                
                chosenDF = foodSet.loc[foodSet['clusterLabel'] != totFoods.iloc[0]['clusterLabel']]
    
                print("식품 갯수 : ", len(chosenDF))           
                print("부족분 : ", abs(float(nutriNeedsDF[nutri])))
                chosenDF = chosenDF.loc[foodSet[nutri] >= abs(float(nutriNeedsDF[nutri]))]
                
                print(chosenDF.head())        
    
            
            # 충족 음식 중 최소값 선정(과식 방지)
            print("충족 음식 중 최솟값 : ", chosenDF[nutri].min())
            print("충족 음식 중 최댓값 : ", chosenDF[nutri].max())
            chosenDF = chosenDF.loc[chosenDF[nutri] == chosenDF[nutri].min()]
            
            # 추천 음식이 다수일 경우를 위한 조치
            for idx in range(0, len(chosenDF)):
                
                print('추천 음식 : %s\n' % chosenDF.iloc[idx])        
                chosenFood = chosenDF.iloc[idx]['식품명']
                print('%s 부족분에 대한 추천 음식(들) : %s' % (nutri, chosenFood))
                print("-" * 100 + "\n")
            
        else: # 영양소 과잉에 대한 메시징
    
            print("현재 섭취하신 음식에는 %s 이(가) 권장 기준량보다 %.2f 만큼 과다합니다. 추후 조절하여 섭취하시기 바랍니다."  \
                  % (nutri, float(nutriNeedsDF[nutri])))
    
            print("*" * 120)    
            
            
######## 질병별 특별 추천 #############################################################

if disease == '암':        

     for idx in range(0, len(cancerFoodDF)):
                
        chosenFood = cancerFoodDF.iloc[idx]['식품명']
        print('암환자를 위한 추천 음식(들) : %s' % chosenFood)
        print("-" * 100 + "\n")

elif disease == '고혈압':
    
    for idx in range(0, len(hypertensionFoodDF)):
             
        chosenFood = hypertensionFoodDF.iloc[idx]['식품명']
        print('고혈압 환자를 위한 대한 특별 추천 음식(들) : %s' % chosenFood)
        print("-" * 100 + "\n")

elif disease == '당뇨':        
    
    print("당뇨 환자의 경우")

    print("*" * 120)  

    for nutri in nutriNeedsDF:
            
        if float(nutriNeedsDF[nutri]) < 0: # 결핍 영양소
        
            # 결핍 영양소에 따른 음식 검색
            # 입력된 음식들의 군집 범위 내에서 검색하되 첫번째 음식부터 검색
            print("부족 영양소(nutri) : ", nutri)
            print("영양소 부족분(절대값) : ", abs(float(nutriNeedsDF[nutri])))
    
            # foodSet에서 같은 클러스터의 권장 섭취분 해당 음식 검색
    
            # 과식 방지를 위해 부족분(ex. 단백질 -15.63 ==> 15.63)에 가장 가까운 값(최소값) 선정
            # 첫번째 입력 음식의 clusterLabel과 비교
            print("clusterLabel : ", totFoods.iloc[0]['clusterLabel'])

            print(diabetesFoodDF[nutri].head())
            
            # 영양소 결핍분에 대한 충족 음식 조회
            chosenDF = diabetesFoodDF.loc[diabetesFoodDF[nutri] >= abs(float(nutriNeedsDF[nutri]))]

            #print('추천 : ', chosenDF.head())
            #print('추천 갯수 : ', len(chosenDF))
    
            # 추천할 것이 없을 경우
            if chosenDF.empty == True:

                # print('추천할 것이 없을 경우')
                
                chosenDF = diabetesFoodDF.loc[diabetesFoodDF[nutri] <= abs(float(nutriNeedsDF[nutri]))]
                print("충족 음식 중 최댓값 : ", chosenDF[nutri].max())
                chosenDF = chosenDF.loc[chosenDF[nutri] == chosenDF[nutri].max()]
                
                print("식품 갯수 : ", len(chosenDF))           
                print("부족분 : ", abs(float(nutriNeedsDF[nutri])))
                #chosenDF = chosenDF.loc[diabetesFoodDF[nutri] <= abs(float(nutriNeedsDF[nutri]))]

                print(chosenDF.head())  
                
            else :
                # 충족 음식 중 최소값 선정(과식 방지)
                print("충족 음식 중 최솟값 : ", chosenDF[nutri].min())
                print("충족 음식 중 최댓값 : ", chosenDF[nutri].max())
                chosenDF = chosenDF.loc[chosenDF[nutri] == chosenDF[nutri].min()]

                print(chosenDF.head()) 
            
                # 추천 음식이 다수일 경우를 위한 조치
                for idx in range(0, len(chosenDF)):
                    
                    print('추천 음식 : %s\n' % chosenDF.iloc[idx])        
                    chosenFood = chosenDF.iloc[idx]['식품명']
                    print('%s 부족분에 대한 추천 음식(들) : %s' % (nutri, chosenFood))
                    print("-" * 100 + "\n")
            
        else: # 영양소 과잉에 대한 메시징
    
            print("현재 섭취하신 음식에는 %s 이(가) 권장 기준량보다 %.2f 만큼 과다합니다. 추후 조절하여 섭취하시기 바랍니다."  \
                  % (nutri, float(nutriNeedsDF[nutri])))
    
            #print("*" * 120)

        print("\n" + "#" * 120 + "\n")


암환자를 위한 추천 음식(들) : 환자용 식품_뉴케어 인핸서 오리지널 CLASSIC
----------------------------------------------------------------------------------------------------

암환자를 위한 추천 음식(들) : 환자용 식품_에브리데이 올케어 스틱
----------------------------------------------------------------------------------------------------

암환자를 위한 추천 음식(들) : 환자용 식품_케어밀 고단백
----------------------------------------------------------------------------------------------------



In [25]:
cursor.close()
conn.close()                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       